# 23章　訓練済みモデルのセーブとロード 

## レシピ23.1　scikit-learnモデルのセーブとロード 


In [ ]:
# ライブラリをロード
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets

# データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

# ランダムフォレストクラス分類器を作成
classifer = RandomForestClassifier()

# モデルを訓練
model = classifer.fit(features, target)

# pickleファイルとしてセーブ
joblib.dump(model, "model.pkl")

In [ ]:
# モデルをファイルからロード
classifer = joblib.load("model.pkl")

# 新たな観測値を作成
new_observation = [[ 5.2, 3.2, 1.1, 0.1]]

# 観測値のクラスを予測
classifer.predict(new_observation)

In [ ]:
# ライブラリをロード
import sklearn

# scikit-learnのバージョンを取得
scikit_version = sklearn.__version__

# pickleファイルとしてセーブ
joblib.dump(model, "model_{version}.pkl".format(version=scikit_version))

## レシピ23.2　TensorFlowモデルのセーブとロード 


In [ ]:
# ライブラリをロード
import numpy as np
from tensorflow import keras
import tensorflow as tf

# 乱数シードを設定
np.random.seed(0)

# 隠れ層1層を持つモデルを作成
input_layer = keras.Input(shape=(10,))
hidden_layer = keras.layers.Dense(10)(input_layer)
output_layer = keras.layers.Dense(1)(input_layer)
model = keras.Model(input_layer, output_layer)
model.compile(optimizer="adam", loss="mean_squared_error")

# モデルを訓練
x_train = np.random.random((1000, 10))
y_train = np.random.random((1000, 1))
model.fit(x_train, y_train)

# モデルを`saved_model`というディレクトリにセーブ
model.export("saved_model")

In [ ]:
# ニューラルネットワークをロード
model = tf.saved_model.load("saved_model")

## レシピ23.3　PyTorchモデルのセーブとロード 


In [ ]:
# ライブラリをロード
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 訓練セットとテストセットを作成
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 乱数シードを設定
torch.manual_seed(0)
np.random.seed(0)

# データをPyTorchのテンソルに変換
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# `Sequential`を用いてニューラルネットワークを定義
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Dropout(0.1), # ニューロンの10%をドロップ
            torch.nn.Sigmoid(),
        )
    def forward(self, x):
        x = self.sequential(x)
        return x

# ニューラルネットワークを初期化
network = SimpleNeuralNet()

# ロス関数と最適化器を定義
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# データローダを定義
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# torch 2.0の最適化器を用いてモデルをコンパイル
network = torch.compile(network)

# ニューラルネットワークを訓練
epochs = 5
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# 訓練したモデルをセーブ
torch.save(
    {
        'epoch': epoch,
        'model_state_dict': network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    },
    "model.pt"
)

# ニューラルネットワークを再初期化
network = SimpleNeuralNet()
state_dict = torch.load(
    "model.pt",
    map_location=torch.device('cpu')
)["model_state_dict"]
network.load_state_dict(state_dict, strict=False)
network.eval()

## レシピ23.4　scikit-learnモデルのサービス化 


In [ ]:
# ライブラリをロード
import joblib
from flask import Flask, request

# Flaskアプリケーションオブジェクトを作成
app = Flask(__name__)

# ディスクからモデルをロード
model = joblib.load("model.pkl")

# predictというrouteを作成する。JSONデータを受け取り、予測を行い、結果を返す。
@app.route("/predict", methods = ["POST"])
def predict():
    print(request.json)
    inputs = request.json["inputs"]
    prediction = model.predict(inputs)
    return {
        "prediction" : prediction.tolist()
    }

# アプリケーションを実行
if __name__ == "__main__":
    app.run()


## レシピ23.5　TensorFlowモデルのサービス化 


## レシピ23.6　 Seldonを用いたPyTorchモデルのWebサービス化 


In [ ]:
# ライブラリをロード
import torch
import torch.nn as nn
import logging

# PyTorchのモデルクラスを作成
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Dropout(0.1), # Drop 10% of neurons
            torch.nn.Sigmoid(),
        )

# Seldonのモデルオブジェクトを`MyModel`という名前で作成
class MyModel(object):
    # モデルのロード
    def __init__(self):
        self.network = SimpleNeuralNet()
        self.network.load_state_dict(
        torch.load("model.pt")["model_state_dict"],
            strict=False
        )
        logging.info(self.network.eval())
    # 予測
    def predict(self, X, features_names=None):
        return self.network.forward(X)
    